In [3]:
%pwd

'/home/mw/project'

In [10]:
%load_ext klab-autotime

The klab-autotime extension is already loaded. To reload it, use:
  %reload_ext klab-autotime
time: 1.33 ms


In [11]:
import kabuki
import hddm
import matplotlib.pyplot as plt
import pickle
from functools import partial
from p_tqdm import p_map
from glob import glob

time: 991 µs


In [5]:
data = hddm.load_csv('/home/mw/input/cpp7572/data.csv')
data.rename(columns={'response_time':'rt','response_corr':'response','subject':'subj_idx'},inplace = True)
data['subj_idx']=data['subj_idx'].str.replace(r'sub-0',r'0')
data = data.drop(data[data['cpp_peak']==100].index)
data

,rt,response,subj_idx,attention,coherency,prioritization,category,cpp_peak,cpp_slope,cpp_amplitude
0,0.421184,1,017,right,low,yes,face,1.337846e-06,0.000010,1.785131e-06
1,0.327997,1,017,double,high,no,face,1.104632e-06,0.000047,7.713533e-08
2,0.230964,1,017,right,high,yes,face,4.495879e-06,0.000007,3.170688e-06
3,0.284764,1,017,double,low,no,car,1.956533e-06,-0.000022,-2.168577e-06
4,0.516040,1,017,double,low,no,car,1.842128e-06,-0.000060,-3.771772e-06
5,0.377975,1,017,left,low,yes,car,7.198718e-07,0.000021,-1.114468e-06
6,0.388672,0,017,double,low,no,face,4.328616e-06,-0.000001,6.473992e-06
7,0.311690,1,017,right,high,yes,car,4.530776e-06,0.000019,-4.726571e-07
8,0.383053,0,017,double,low,no,face,1.964945e-05,-0.000081,5.106468e-06
9,0.251344,1,017,left,high,yes,face,5.402271e-06,0.000065,7.108547e-06


time: 48.5 ms


In [8]:
def run_m6(id, df=None, samples=None, burn=None, save_name=None): 
#     print('running model %i'%id);
    import hddm
    
    dbname = save_name + '_chain_%i.db'%id 
    mname  = save_name + '_chain_%i'%id    
    v_reg = {'model':'v~1+C(coherency, Treatment("low"))','link_func':lambda x:x}
    a_reg = {'model':'a~1+C(prioritization, Treatment("no"))','link_func':lambda x:x}
    m = hddm.HDDMRegressor(data, [v_reg, a_reg],include=['st','sv','sz'], group_only_regressors = False)
    m.find_starting_values()
    m.sample(samples, burn=burn, dbname=dbname, db='pickle') # it's neccessary to save the model data
    m.save(mname)
    
    return m

time: 1.53 ms


In [9]:
models_6 = p_map(partial(run_m6, 
                         df = data, 
                         samples = 2000,
                         burn = 500,
                         save_name = '/home/mw/project/m6'),
                 range(8))

KeyboardInterrupt: 

time: 11 s


In [12]:
models_6_comb = []
for model_path in glob('/home/mw/project/m6_chain_*'):
    if ('db' not in model_path) and ('csv' not in model_path):
        print(model_path)
        model6 = hddm.load(model_path)
        models_6_comb.append(model6)

/home/mw/project/m4_chain_6
/home/mw/project/m4_chain_2
/home/mw/project/m4_chain_0
/home/mw/project/m4_chain_4
/home/mw/project/m4_chain_7
/home/mw/project/m4_chain_3
/home/mw/project/m4_chain_1
/home/mw/project/m4_chain_5
time: 17.8 s


In [13]:
m6 = kabuki.utils.concat_models(models_6_comb)

time: 2.63 s


In [14]:
m6.gen_stats()

,mean,std,2.5q,25q,50q,75q,97.5q,mc err
a,1.24308,0.125842,1.03252,1.15978,1.23253,1.31162,1.52377,0.00220016
a_std,0.349505,0.12263,0.194668,0.267944,0.324992,0.399786,0.648908,0.00226028
a_subj.001,1.24798,0.10521,1.06837,1.17426,1.23857,1.31213,1.4804,0.00264247
a_subj.003,1.3321,0.079757,1.1895,1.27586,1.32797,1.38305,1.4981,0.00163034
a_subj.005,1.08344,0.0758728,0.946225,1.03067,1.07909,1.1319,1.24391,0.0013983
a_subj.006,1.72635,0.132708,1.50049,1.63322,1.71555,1.80787,2.01164,0.00342118
a_subj.008,1.15509,0.0947997,0.986191,1.08848,1.1486,1.2143,1.35875,0.00219895
a_subj.010,0.729404,0.0504317,0.634188,0.694672,0.727398,0.762509,0.831404,0.000948101
a_subj.012,1.04797,0.0745732,0.914413,0.997798,1.04389,1.09326,1.20708,0.00167552
a_subj.015,1.0267,0.0601203,0.91645,0.985385,1.02376,1.06498,1.15303,0.00116676


time: 308 ms


In [15]:
m6.get_traces().to_csv('/home/mw/temp/m4_trace.csv',index=False)

time: 1.38 s


In [16]:
m6.dic

-2202.313578025329

time: 78.6 ms
